In [ ]:
# gvp: implement gvp on pdb dataset for residue analysis


In [1]:
from constants import AA3_TO_AA1

In [6]:
from pdbHelper import get_atoms_neighbor

In [10]:
# 
import numpy as np
import torch
import torch.nn as nn
from gvp import GVP, GVPConvLayer, LayerNorm

In [11]:
GVP

gvp.GVP

In [12]:
torch.cuda.is_available()

False

# get atom3d res dataset

In [1]:
import gvp
from atom3d.datasets import LMDBDataset
import torch_geometric
from functools import partial
import gvp.atom3d
import torch.nn as nn
import tqdm, torch, time, os
import numpy as np
from atom3d.util import metrics
import sklearn.metrics as sk_metrics
from collections import defaultdict
import scipy.stats as stats
print = partial(print, flush=True)

models_dir = 'models'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_id = float(time.time())

In [2]:
print(device)

cuda


In [18]:
lba_split = 30
def get_datasets(task, lba_split=lba_split):
    data_path = {
        'RES' : '/n/groups/marks/users/david/res/atom3d_data/raw/RES/data/',
    }[task]

    if task == 'RES':
        split_path = '/n/groups/marks/users/david/res/atom3d_data/split-by-cath-topology/indices/'
        dataset = partial(gvp.atom3d.RESDataset, data_path)        
        trainset = dataset(split_path=split_path+'train_indices.txt')
        valset = dataset(split_path=split_path+'val_indices.txt')
        testset = dataset(split_path=split_path+'test_indices.txt')


    return trainset, valset, testset

datasets = get_datasets('RES')

In [19]:
datasets

(<gvp.atom3d.RESDataset at 0x7f09142c21d0>,
 <gvp.atom3d.RESDataset at 0x7f08ccb5ca90>)

In [20]:
batch_size = 1 # control memory of model as long as you can fit a batch size of 1, you can do gradient accumultion to simulate batch size.
num_workers = 4
dataloader = partial(torch_geometric.data.DataLoader, 
                    num_workers=num_workers, batch_size=batch_size)

In [6]:
trainset, valset, testset = map(dataloader, datasets)   

/n/groups/marks/software/anaconda_o2/envs/dd_torch/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
#need to create a DataBatch with the following attributes:
(x=[531, 3], # xyz of each atom
edge_index=[2, 8336], # connections between each atom
 atoms=[531], # [2, 1, 1, 3, 1, 2, 1, 1, ], presumably codes for which atom it is
 edge_s=[8336, 16], # edge scalar features, 16 numbers for each atom
 edge_v=[8336, 1, 3], # edge vectors for each edge
 label=[1], # just the amino acid label
 ca_idx=[1], # just says [1], or [16].presumably theindex
 batch=[531], #just all indicates atoms which belong to one batch [0, *len(atoms)
 ptr=[2], #tensor([0,531]) # not sure what

In [25]:
c=0
for b in testset:
    if c<1:
        print(b)
        print(b.batch)
        print(b.batch.shape)
        print(b.ptr)
        print(b.label)
        print('ca_idx', b.ca_idx)
        print('atoms',b.atoms)
        #out = forward(model, b, device)
        #print(out.shape)
        #print(out)
        c+=1
    else:
        break



DataBatch(x=[452, 3], edge_index=[2, 7012], atoms=[452], edge_s=[7012, 16], edge_v=[7012, 1, 3], label=[1], ca_idx=[1], batch=[452], ptr=[2])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# start training from beginning

In [7]:
def get_model(task):
    return {
        'RES' : gvp.atom3d.RESModel,
    }[task]()

model = get_model('RES').to(device)

In [8]:
'''
parser.add_argument('--num-workers', metavar='N', type=int, default=4,
                   help='number of threads for loading data, default=4')
parser.add_argument('--smp-idx', metavar='IDX', type=int, default=None,
                   choices=list(range(20)),
                   help='label index for SMP, in range 0-19')
parser.add_argument('--lba-split', metavar='SPLIT', type=int, choices=[30, 60],
                    help='identity cutoff for LBA, 30 (default) or 60', default=30)
parser.add_argument('--batch', metavar='SIZE', type=int, default=8,
                    help='batch size, default=8')
parser.add_argument('--train-time', metavar='MINUTES', type=int, default=120,
                    help='maximum time between evaluations on valset, default=120 minutes')
parser.add_argument('--val-time', metavar='MINUTES', type=int, default=20,
                    help='maximum time per evaluation on valset, default=20 minutes')
parser.add_argument('--epochs', metavar='N', type=int, default=50,
                    help='training epochs, default=50')
parser.add_argument('--test', metavar='PATH', default=None,
                    help='evaluate a trained model')
parser.add_argument('--lr', metavar='RATE', default=1e-4, type=float,
                    help='learning rate')
parser.add_argument('--load', metavar='PATH', default=None, 
                    help='initialize first 2 GNN layers with pretrained weights')
'''

lr = 1e-4
epochs = 50
train_time = 120
val_time = 20
model_save_path = '/n/groups/marks/users/david/res/model_save/'
smp_idx = None


def get_label(batch, task, smp_idx=None):
    if type(batch) in [list, tuple]: batch = batch[0]
    if task == 'SMP':
        assert smp_idx is not None
        return batch.label[smp_idx::20]
    return batch.label

def forward(model, batch, device):
    if type(batch) in [list, tuple]:
        batch = batch[0].to(device), batch[1].to(device)
    else:
        batch = batch.to(device)
    return model(batch)

def loop(dataset, model, optimizer=None, max_time=None):
    start = time.time()
    
    loss_fn = nn.CrossEntropyLoss()
    t = tqdm.tqdm(dataset)
    total_loss, total_count = 0, 0
    
    for batch in t:
        if max_time and (time.time() - start) > 60*max_time: break
        if optimizer: optimizer.zero_grad()
        try:
            out = forward(model, batch, device)
        except RuntimeError as e:
            if "CUDA out of memory" not in str(e): raise(e)
            torch.cuda.empty_cache()
            print('Skipped batch due to OOM', flush=True)
            continue
            
        label = get_label(batch, 'RES', smp_idx)
        loss_value = loss_fn(out, label)
        total_loss += float(loss_value)
        total_count += 1
        
        if optimizer:
            try:
                loss_value.backward()
                optimizer.step()
            except RuntimeError as e:
                if "CUDA out of memory" not in str(e): raise(e)
                torch.cuda.empty_cache()
                print('Skipped batch due to OOM', flush=True)
                continue
            
        t.set_description(f"{total_loss/total_count:.8f}")
        
    return total_loss / total_count

def train(model, trainset, valset):

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_path, best_val = None, np.inf

    for epoch in range(epochs):
        model.train()
        loss = loop(trainset, model, optimizer=optimizer, max_time=train_time)
        path = f"{model_save_path}/RES_{model_id}_{epoch}.pt"
        torch.save(model.state_dict(), path)
        print(f'\nEPOCH {epoch} TRAIN loss: {loss:.8f}')
        model.eval()
        with torch.no_grad():
            loss = loop(valset, model, max_time=val_time)
        print(f'\nEPOCH {epoch} VAL loss: {loss:.8f}')
        if loss < best_val:
            best_path, best_val = path, loss
        print(f'BEST {best_path} VAL loss: {best_val:.8f}')

        

In [9]:
train(model, trainset, valset)

2.99179641: : 2541it [15:03,  2.81it/s]


KeyboardInterrupt: 

# start training from loaded best model

In [9]:
def load(model, path):
    # this only initializes the gnn with pretrained wweights for first 2 layers.
    params = torch.load(path)
    state_dict = model.state_dict()
    for name, p in params.items():
        if name in state_dict and \
               name[:8] in ['layers.0', 'layers.1'] and \
               state_dict[name].shape == p.shape:
            print("Loading", name)
            model.state_dict()[name].copy_(p)

def get_model(task):
    return {
        'RES' : gvp.atom3d.RESModel,
    }[task]()


load_path = '/n/groups/marks/users/david/res/model_save/RES_1638990570.3313198_32.pt'

model = get_model('RES').to(device)
load(model, load_path)

Loading layers.0.conv.message_func.0.dummy_param
Loading layers.0.conv.message_func.0.wh.weight
Loading layers.0.conv.message_func.0.ws.weight
Loading layers.0.conv.message_func.0.ws.bias
Loading layers.0.conv.message_func.0.wv.weight
Loading layers.0.conv.message_func.0.wsv.weight
Loading layers.0.conv.message_func.0.wsv.bias
Loading layers.0.conv.message_func.1.dummy_param
Loading layers.0.conv.message_func.1.wh.weight
Loading layers.0.conv.message_func.1.ws.weight
Loading layers.0.conv.message_func.1.ws.bias
Loading layers.0.conv.message_func.1.wv.weight
Loading layers.0.conv.message_func.1.wsv.weight
Loading layers.0.conv.message_func.1.wsv.bias
Loading layers.0.conv.message_func.2.dummy_param
Loading layers.0.conv.message_func.2.wh.weight
Loading layers.0.conv.message_func.2.ws.weight
Loading layers.0.conv.message_func.2.ws.bias
Loading layers.0.conv.message_func.2.wv.weight
Loading layers.0.conv.message_func.2.wsv.weight
Loading layers.0.conv.message_func.2.wsv.bias
Loading layer

In [12]:

def get_metrics():
    return {'accuracy': metrics.accuracy}

def test(model, testset, model_path):
    #model.load_state_dict(torch.load(args.test))
    model.load_state_dict(torch.load(model_path))

    model.eval()
    t = tqdm.tqdm(testset)
    metrics = get_metrics()
    targets, predicts, ids = [], [], []
    with torch.no_grad():
        for batch in t:
            pred = forward(model, batch, device)
            label = get_label(batch, 'RES', None)
            pred = pred.argmax(dim=-1)

            targets.extend(list(label.cpu().numpy()))
            predicts.extend(list(pred.cpu().numpy()))

    for name, func in metrics.items():

        value = func(targets, predicts)
        print(f"{name}: {value}")

test_path = '/n/groups/marks/users/david/res/model_save/RES_1638990570.3313198_32.pt'

test(model, testset, test_path)

74334it [2:48:21,  7.36it/s]


accuracy: 0.3937415707285245


In [11]:
train(model, trainset, valset)

2.99543196: : 8it [00:03,  2.14it/s]


KeyboardInterrupt: 

# evaluate one batch

In [13]:
c=0
for b in testset:
    if c<2:
        print(b)
        out = forward(model, b, device)
        print(out.shape)
        print(out)
        c+=1
    

DataBatch(x=[4885, 3], edge_index=[2, 75300], atoms=[4885], edge_s=[75300, 16], edge_v=[75300, 1, 3], label=[8], ca_idx=[8], batch=[4885], ptr=[9])
torch.Size([8, 20])
tensor([[-0.0790,  0.0710,  0.0800,  0.0177,  0.1705,  0.0948, -0.0083,  0.0108,
         -0.0270,  0.0615,  0.0171,  0.0041,  0.0731,  0.0400,  0.0094,  0.0365,
         -0.1175, -0.0768, -0.0051, -0.0794],
        [-0.0865,  0.1662, -0.0075, -0.0152,  0.2250,  0.1340, -0.0011,  0.0296,
          0.0376,  0.0513,  0.0471,  0.0281,  0.0631,  0.0414,  0.0320,  0.0798,
         -0.0887, -0.1366, -0.0847, -0.0250],
        [-0.0836,  0.0540,  0.0403,  0.0193,  0.1484,  0.0747, -0.0473,  0.0288,
         -0.0489, -0.0130, -0.0221, -0.0369,  0.0947,  0.0056, -0.0653,  0.1089,
         -0.0889, -0.1099, -0.0373, -0.1150],
        [-0.0770,  0.1295,  0.0805, -0.0212,  0.1863,  0.1097,  0.0389,  0.0483,
         -0.0256,  0.0233, -0.0051, -0.0077,  0.0711,  0.0072,  0.0035,  0.0318,
         -0.0786, -0.1463,  0.0223, -0.0726],


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f0a76eeba70>
Traceback (most recent call last):
  File "/n/groups/marks/software/anaconda_o2/envs/dd_torch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/n/groups/marks/software/anaconda_o2/envs/dd_torch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/n/groups/marks/software/anaconda_o2/envs/dd_torch/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f0a76eeba70>
Traceback (most recent call last):
  File "/n/groups/marks/software/anaconda_o2/envs/dd_torch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown

# misc

In [13]:
# from gvp-pytorch: run_atom3d.py
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('task', metavar='TASK', choices=[
        'PSR', 'RSR', 'PPI', 'RES', 'MSP', 'SMP', 'LBA', 'LEP'
    ], help="{PSR, RSR, PPI, RES, MSP, SMP, LBA, LEP}")
parser.add_argument('--num-workers', metavar='N', type=int, default=4,
                   help='number of threads for loading data, default=4')
parser.add_argument('--smp-idx', metavar='IDX', type=int, default=None,
                   choices=list(range(20)),
                   help='label index for SMP, in range 0-19')
parser.add_argument('--lba-split', metavar='SPLIT', type=int, choices=[30, 60],
                    help='identity cutoff for LBA, 30 (default) or 60', default=30)
parser.add_argument('--batch', metavar='SIZE', type=int, default=8,
                    help='batch size, default=8')
parser.add_argument('--train-time', metavar='MINUTES', type=int, default=120,
                    help='maximum time between evaluations on valset, default=120 minutes')
parser.add_argument('--val-time', metavar='MINUTES', type=int, default=20,
                    help='maximum time per evaluation on valset, default=20 minutes')
parser.add_argument('--epochs', metavar='N', type=int, default=50,
                    help='training epochs, default=50')
parser.add_argument('--test', metavar='PATH', default=None,
                    help='evaluate a trained model')
parser.add_argument('--lr', metavar='RATE', default=1e-4, type=float,
                    help='learning rate')
parser.add_argument('--load', metavar='PATH', default=None, 
                    help='initialize first 2 GNN layers with pretrained weights')

args = parser.parse_args()



def main():
    datasets = get_datasets(args.task, args.lba_split)
    dataloader = partial(torch_geometric.data.DataLoader, 
                    num_workers=args.num_workers, batch_size=args.batch)
    if args.task not in ['PPI', 'RES']:
        dataloader = partial(dataloader, shuffle=True)
        
    trainset, valset, testset = map(dataloader, datasets)    
    model = get_model(args.task).to(device)
    
    if args.test:
        test(model, testset)

    else:
        if args.load:
            load(model, args.load)
        train(model, trainset, valset)
        
def test(model, testset):
    model.load_state_dict(torch.load(args.test))
    model.eval()
    t = tqdm.tqdm(testset)
    metrics = get_metrics(args.task)
    targets, predicts, ids = [], [], []
    with torch.no_grad():
        for batch in t:
            pred = forward(model, batch, device)
            label = get_label(batch, args.task, args.smp_idx)
            if args.task == 'RES':
                pred = pred.argmax(dim=-1)
            if args.task in ['PSR', 'RSR']:
                ids.extend(batch.id)
            targets.extend(list(label.cpu().numpy()))
            predicts.extend(list(pred.cpu().numpy()))

    for name, func in metrics.items():
        if args.task in ['PSR', 'RSR']:
            func = partial(func, ids=ids)
        value = func(targets, predicts)
        print(f"{name}: {value}")

def train(model, trainset, valset):
                                
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    
    best_path, best_val = None, np.inf
    
    for epoch in range(args.epochs):
        model.train()
        loss = loop(trainset, model, optimizer=optimizer, max_time=args.train_time)
        path = f"{models_dir}/{args.task}_{model_id}_{epoch}.pt"
        torch.save(model.state_dict(), path)
        print(f'\nEPOCH {epoch} TRAIN loss: {loss:.8f}')
        model.eval()
        with torch.no_grad():
            loss = loop(valset, model, max_time=args.val_time)
        print(f'\nEPOCH {epoch} VAL loss: {loss:.8f}')
        if loss < best_val:
            best_path, best_val = path, loss
        print(f'BEST {best_path} VAL loss: {best_val:.8f}')
    
def loop(dataset, model, optimizer=None, max_time=None):
    start = time.time()
    
    loss_fn = get_loss(args.task)
    t = tqdm.tqdm(dataset)
    total_loss, total_count = 0, 0
    
    for batch in t:
        if max_time and (time.time() - start) > 60*max_time: break
        if optimizer: optimizer.zero_grad()
        try:
            out = forward(model, batch, device)
        except RuntimeError as e:
            if "CUDA out of memory" not in str(e): raise(e)
            torch.cuda.empty_cache()
            print('Skipped batch due to OOM', flush=True)
            continue
            
        label = get_label(batch, args.task, args.smp_idx)
        loss_value = loss_fn(out, label)
        total_loss += float(loss_value)
        total_count += 1
        
        if optimizer:
            try:
                loss_value.backward()
                optimizer.step()
            except RuntimeError as e:
                if "CUDA out of memory" not in str(e): raise(e)
                torch.cuda.empty_cache()
                print('Skipped batch due to OOM', flush=True)
                continue
            
        t.set_description(f"{total_loss/total_count:.8f}")
        
    return total_loss / total_count

def load(model, path):
    params = torch.load(path)
    state_dict = model.state_dict()
    for name, p in params.items():
        if name in state_dict and \
               name[:8] in ['layers.0', 'layers.1'] and \
               state_dict[name].shape == p.shape:
            print("Loading", name)
            model.state_dict()[name].copy_(p)
        
#######################################################################

def get_label(batch, task, smp_idx=None):
    if type(batch) in [list, tuple]: batch = batch[0]
    if task == 'SMP':
        assert smp_idx is not None
        return batch.label[smp_idx::20]
    return batch.label

def get_metrics(task):
    def _correlation(metric, targets, predict, ids=None, glob=True):
        if glob: return metric(targets, predict)
        _targets, _predict = defaultdict(list), defaultdict(list)
        for _t, _p, _id in zip(targets, predict, ids):
            _targets[_id].append(_t)
            _predict[_id].append(_p)
        return np.mean([metric(_targets[_id], _predict[_id]) for _id in _targets])
        
    correlations = {
        'pearson': partial(_correlation, metrics.pearson),
        'kendall': partial(_correlation, metrics.kendall),
        'spearman': partial(_correlation, metrics.spearman)
    }
    mean_correlations = {f'mean {k}' : partial(v, glob=False) \
                            for k, v in correlations.items()}
    
    return {                       
        'RSR' : {**correlations, **mean_correlations},
        'PSR' : {**correlations, **mean_correlations},
        'PPI' : {'auroc': metrics.auroc},
        'RES' : {'accuracy': metrics.accuracy},
        'MSP' : {'auroc': metrics.auroc, 'auprc': metrics.auprc},
        'LEP' : {'auroc': metrics.auroc, 'auprc': metrics.auprc},
        'LBA' : {**correlations, 'rmse': partial(sk_metrics.mean_squared_error, squared=False)},
        'SMP' : {'mae': sk_metrics.mean_absolute_error}
    }[task]
            
def get_loss(task):
    if task in ['PSR', 'RSR', 'SMP', 'LBA']: return nn.MSELoss() # regression
    elif task in ['PPI', 'MSP', 'LEP']: return nn.BCELoss() # binary classification
    elif task in ['RES']: return nn.CrossEntropyLoss() # multiclass classification
    
def forward(model, batch, device):
    if type(batch) in [list, tuple]:
        batch = batch[0].to(device), batch[1].to(device)
    else:
        batch = batch.to(device)
    return model(batch)

def get_datasets(task, lba_split=30):
    data_path = {
        'RES' : 'atom3d-data/RES/raw/RES/data/',
        'PPI' : 'atom3d-data/PPI/splits/DIPS-split/data/',
        'RSR' : 'atom3d-data/RSR/splits/candidates-split-by-time/data/',
        'PSR' : 'atom3d-data/PSR/splits/split-by-year/data/',
        'MSP' : 'atom3d-data/MSP/splits/split-by-sequence-identity-30/data/',
        'LEP' : 'atom3d-data/LEP/splits/split-by-protein/data/',
        'LBA' : f'atom3d-data/LBA/splits/split-by-sequence-identity-{lba_split}/data/',
        'SMP' : 'atom3d-data/SMP/splits/random/data/'
    }[task]
        
    if task == 'RES':
        split_path = 'atom3d-data/RES/splits/split-by-cath-topology/indices/'
        dataset = partial(gvp.atom3d.RESDataset, data_path)        
        trainset = dataset(split_path=split_path+'train_indices.txt')
        valset = dataset(split_path=split_path+'val_indices.txt')
        testset = dataset(split_path=split_path+'test_indices.txt')
    
    elif task == 'PPI':
        trainset = gvp.atom3d.PPIDataset(data_path+'train')
        valset = gvp.atom3d.PPIDataset(data_path+'val')
        testset = gvp.atom3d.PPIDataset(data_path+'test')
        
    else:
        transform = {                       
            'RSR' : gvp.atom3d.RSRTransform,
            'PSR' : gvp.atom3d.PSRTransform,
            'MSP' : gvp.atom3d.MSPTransform,
            'LEP' : gvp.atom3d.LEPTransform,
            'LBA' : gvp.atom3d.LBATransform,
            'SMP' : gvp.atom3d.SMPTransform,
        }[task]()
        
        trainset = LMDBDataset(data_path+'train', transform=transform)
        valset = LMDBDataset(data_path+'val', transform=transform)
        testset = LMDBDataset(data_path+'test', transform=transform)
        
    return trainset, valset, testset

def get_model(task):
    return {
        'RES' : gvp.atom3d.RESModel,
    }[task]()

main()

usage: ipykernel_launcher.py [-h] [--num-workers N] [--smp-idx IDX]
                             [--lba-split SPLIT] [--batch SIZE]
                             [--train-time MINUTES] [--val-time MINUTES]
                             [--epochs N] [--test PATH] [--lr RATE]
                             [--load PATH]
                             TASK
ipykernel_launcher.py: error: argument TASK: invalid choice: '/home/dd128/.local/share/jupyter/runtime/kernel-970b75c3-b998-4365-a189-164295414e77.json' (choose from 'PSR', 'RSR', 'PPI', 'RES', 'MSP', 'SMP', 'LBA', 'LEP')
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/n/groups/marks/software/anaconda_o2/envs/dd_torch/lib/python3.7/argparse.py", line 1787, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/n/groups/marks/software/anaconda_o2/envs/dd_torch/lib/python3.7/argparse.py", line 1996, in _parse_known_args
    stop_index = consume_positionals(start_index)
  File "/n/groups/marks/software/anaconda_o2/envs/dd_torch/lib/python3.7/argparse.py", line 1952, in consume_positionals
    take_action(action, args)
  File "/n/groups/marks/software/anaconda_o2/envs/dd_torch/lib/python3.7/argparse.py", line 1845, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "/n/groups/marks/software/anaconda_o2/envs/dd_torch/lib/python3.7/argparse.py", line 2377, in _get_values
    self._check_value(action, value)
  File "/n/groups/marks/software/anaconda_o2/envs/dd_torch/lib/python3.7/argparse.py", line 2433, in _check_value
    raise ArgumentError

TypeError: object of type 'NoneType' has no len()